In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv('Trials.csv')

In [3]:
df.head(2)

,id,name,screen_name,fav_number,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,...,profile_sidebar_fill_color,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,protected,verified,description,updated,dataset
0,3610511,Davide Dellacasa,braddd,0,20370,5470,2385,145,52,Fri Apr 06 10:58:22 +0000 2007,...,FFF7CC,http://a0.twimg.com/profile_background_images/...,BADFCD,FF0000,3600.0,NaN,NaN,Founder of http://www.screenweek.it & http://w...,2/14/2015 10:54,1
1,5656162,Simone Economo,eKoeS,68,3131,506,381,9,40,Mon Apr 30 15:08:42 +0000 2007,...,DDEEF6,http://a0.twimg.com/images/themes/theme1/bg.png,C0DEED,0084B4,3600.0,NaN,NaN,BSc degree (cum laude) in Computer Engineering...,2/14/2015 10:54,1


In [4]:
df['dataset'].value_counts()

1    1481
2    1337
Name: dataset, dtype: int64

In [5]:
real_df = df[df['dataset']==1][0:200]
fake_df = df[df['dataset']==2][0:200]

In [6]:
df['created_at'] = pd.to_datetime(df.created_at)

In [7]:
df['Month'] = df.created_at.dt.month

In [8]:
df['Year'] = df.created_at.dt.year

In [9]:
df['Time'] = df.created_at.dt.time

In [10]:
df.head(2)

,id,name,screen_name,fav_number,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,...,profile_link_color,utc_offset,protected,verified,description,updated,dataset,Month,Year,Time
0,3610511,Davide Dellacasa,braddd,0,20370,5470,2385,145,52,2007-04-06 10:58:22+00:00,...,FF0000,3600.0,NaN,NaN,Founder of http://www.screenweek.it & http://w...,2/14/2015 10:54,1,4,2007,10:58:22
1,5656162,Simone Economo,eKoeS,68,3131,506,381,9,40,2007-04-30 15:08:42+00:00,...,0084B4,3600.0,NaN,NaN,BSc degree (cum laude) in Computer Engineering...,2/14/2015 10:54,1,4,2007,15:08:42


# Label Encoding 

In [11]:
from sklearn import preprocessing

In [12]:
le = preprocessing.LabelEncoder()

In [13]:
le.fit(df['screen_name'])

LabelEncoder()

In [14]:
df['screen_name'] = le.transform(df['screen_name'])

In [15]:
le.fit(df['name'])

LabelEncoder()

In [16]:
df['name'] = le.transform(df['name'])

In [17]:
le.fit(df['Time'])

LabelEncoder()

In [18]:
df['Time'] = le.transform(df['Time'])

# Feature Engineering 

In [19]:
df.columns

Index(['id', 'name', 'screen_name', 'fav_number', 'statuses_count',
       'followers_count', 'friends_count', 'favourites_count', 'listed_count',
       'created_at', 'url', 'lang', 'time_zone', 'location', 'default_profile',
       'default_profile_image', 'geo_enabled', 'profile_image_url',
       'profile_banner_url', 'profile_use_background_image',
       'profile_background_image_url_https', 'profile_text_color',
       'profile_image_url_https', 'profile_sidebar_border_color',
       'profile_background_tile', 'profile_sidebar_fill_color',
       'profile_background_image_url', 'profile_background_color',
       'profile_link_color', 'utc_offset', 'protected', 'verified',
       'description', 'updated', 'dataset', 'Month', 'Year', 'Time'],
      dtype='object')

In [55]:
features_df = df[['name', 'screen_name', 'fav_number', 'statuses_count','followers_count', 'friends_count', 'favourites_count', 'listed_count', 'Year', 'Month', 'Time']]

In [56]:
features_df.columns

Index(['name', 'screen_name', 'fav_number', 'statuses_count',
       'followers_count', 'friends_count', 'favourites_count', 'listed_count',
       'Year', 'Month', 'Time'],
      dtype='object')

In [57]:
X = np.asarray(features_df)

In [58]:
y = np.asarray(df['dataset'])

In [59]:
X[0:6]

array([[  573,  1106,     0, 20370,  5470,  2385,   145,    52,  2007,
            4,   763],
       [ 2251,  1409,    68,  3131,   506,   381,     9,    40,  2007,
            4,  1282],
       [ 2767,  2626,  7696,  4024,   264,    87,   323,    16,  2007,
            5,   872],
       [ 2527,   942,   202, 40586,   640,   622,  1118,    32,  2007,
            5,  1880],
       [  157,   680, 37318,  2016,    62,    64,    13,     0,  2007,
            5,  2257],
       [  327,  1291,  3901,  3603,   138,   179,    53,     1,  2007,
            5,  1299]], dtype=int64)

# Train Test Split

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
X_train,X_test, y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 4)

# Importing the XGBoost Model Method 1

In [62]:
import xgboost as xgb

In [63]:
train = xgb.DMatrix(X_train, label= y_train)
test = xgb.DMatrix(X_test, label=y_test)

In [64]:
param = {'max_depth':4, 'eta':0.3,'num_class':3}
epochs = 10

In [65]:
model = xgb.train(param,train, epochs)

In [66]:
df['dataset'].unique()

array([1, 2], dtype=int64)

In [67]:
predictions = model.predict(test)

In [68]:
print(predictions)

[2. 1. 1. 2. 1. 1. 1. 1. 2. 1. 1. 2. 2. 1. 2. 2. 2. 1. 1. 2. 1. 1. 1. 1.
 2. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 2. 2. 1. 2. 2. 1. 1. 1. 2. 1. 2. 1. 1.
 1. 1. 1. 2. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 2. 1. 2. 1. 1. 1. 2. 1. 2. 2.
 2. 1. 2. 1. 1. 2. 1. 2. 1. 1. 1. 2. 1. 1. 1. 1. 2. 1. 2. 1. 1. 2. 1. 1.
 2. 1. 1. 2. 1. 1. 1. 1. 1. 2. 1. 1. 2. 1. 1. 2. 2. 2. 2. 2. 2. 1. 2. 1.
 2. 1. 2. 2. 2. 2. 1. 1. 2. 2. 2. 2. 1. 2. 1. 1. 1. 2. 1. 1. 2. 1. 1. 2.
 2. 1. 1. 2. 2. 1. 2. 2. 2. 2. 1. 2. 1. 1. 2. 1. 2. 2. 2. 2. 1. 2. 2. 2.
 2. 1. 2. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 2.
 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 2. 2. 1. 1. 2. 2. 1. 1. 2. 2.
 1. 1. 2. 1. 2. 2. 1. 1. 2. 2. 2. 1. 2. 1. 2. 2. 2. 2. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 2. 1. 1. 1. 1. 2. 1. 2. 1. 2. 2.
 2. 1. 2. 1. 1. 1. 1. 2. 2. 2. 2. 2. 1. 2. 1. 2. 1. 1. 1. 1. 1. 2. 1. 2.
 1. 2. 2. 1. 2. 2. 1. 2. 1. 1. 1. 2. 2. 2. 1. 1. 1. 1. 1. 1. 2. 1. 1. 2.
 1. 2. 2. 1. 2. 1. 1. 2. 2. 1. 2. 2. 1. 1. 2. 1. 1.

In [69]:
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix 

In [70]:
accuracy_score (y_test, predictions)

0.9911347517730497

In [54]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           1       1.00      0.99      0.99       298
           2       0.99      1.00      0.99       266

    accuracy                           0.99       564
   macro avg       0.99      0.99      0.99       564
weighted avg       0.99      0.99      0.99       564



In [37]:
print(confusion_matrix(y_test, predictions))

[[298   0]
 [  1 265]]


# Importing the XGBoost Model Method 2